# 👨‍💻 NeMo Data Designer: Text-to-Python

#### 📚 What you'll learn

- This notebook demonstrates how to use NeMo Data Designer to create a synthetic data generation pipeline for Python code examples.

- We'll build a system that generates Python code based on natural language instructions, with varying complexity levels and industry focuses.

<br>

> 👋 **IMPORTANT** – Environment Setup
>
> - If you haven't already, follow the instructions in the [README](../../../README.md) to install the necessary dependencies.
>
> - You may need to restart your notebook's kernel after setting up the environment.
> - In this notebook, we assume you have a self-hosted instance of Data Designer up and running.
>
> - For deployment instructions, see the [Installation Options](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/index.html#installation-options) section of the [NeMo Data Designer documentation](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/index.html).


### 📦 Import the essentials

- The `data_designer` module of `nemo_microservices` exposes Data Designer's high-level SDK.

- The `essentials` module provides quick access to the most commonly used objects.


In [ ]:
from nemo_microservices.data_designer.essentials import (
    CategorySamplerParams,
    CodeLang,
    CodeValidatorParams,
    DataDesignerConfigBuilder,
    InferenceParameters,
    LLMCodeColumnConfig,
    LLMJudgeColumnConfig,
    LLMTextColumnConfig,
    ModelConfig,
    NeMoDataDesignerClient,
    SamplerColumnConfig,
    SamplerType,
    Score,
    SubcategorySamplerParams,
    ValidationColumnConfig,
    ValidatorType,
)

### ⚙️ Initialize the NeMo Data Designer Client

- `NeMoDataDesignerClient` is responsible for submitting generation requests to the microservice.


In [ ]:
NEMO_MICROSERVICES_BASE_URL = "http://localhost:8080"

data_designer_client = NeMoDataDesignerClient(base_url=NEMO_MICROSERVICES_BASE_URL)

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see [the model config docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/configure-models.html) for more details).

- By default, the microservice uses [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [ ]:
# This name is set in the microservice deployment configuration.
MODEL_PROVIDER = "nvidiabuild"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/llama-3.3-nemotron-super-49b-v1"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-super-49b-v1"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.6,
            top_p=0.95,
            max_tokens=1024,
            timeout=300,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [ ]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🎲 Adding Sampler Columns

- Sampler columns offer non-LLM based generation of synthetic data.

- They are particularly useful for **steering the diversity** of the generated data, as we demonstrate below.


In [ ]:
# Add industry sector categories
config_builder.add_column(
    SamplerColumnConfig(
        name="industry_sector",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Healthcare",
                "Finance",
                "Technology",
            ],
        ),
    ),
)

# Add topic as a subcategory of industry_sector
config_builder.add_column(
    SamplerColumnConfig(
        name="topic",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="industry_sector",
            values={
                "Healthcare": [
                    "Electronic Health Records (EHR) Systems",
                    "Telemedicine Platforms",
                    "AI-Powered Diagnostic Tools",
                ],
                "Finance": [
                    "Fraud Detection Software",
                    "Automated Trading Systems",
                    "Personal Finance Apps",
                ],
                "Technology": [
                    "Cloud Computing Platforms",
                    "Artificial Intelligence and Machine Learning Platforms",
                    "DevOps and CI/CD Tools",
                ],
            },
        ),
    ),
)

# Add code complexity with subcategory for code concepts
config_builder.add_column(
    SamplerColumnConfig(
        name="code_complexity",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Beginner",
                "Intermediate",
                "Advanced",
            ],
        ),
    ),
)

# Add code_concept as a subcategory of code_complexity
config_builder.add_column(
    SamplerColumnConfig(
        name="code_concept",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="code_complexity",
            values={
                "Beginner": [
                    "Variables",
                    "Data Types",
                    "Functions",
                    "Loops",
                    "Classes",
                ],
                "Intermediate": [
                    "List Comprehensions",
                    "Object-oriented programming",
                    "Lambda Functions",
                    "Web frameworks",
                    "Pandas",
                ],
                "Advanced": [
                    "Multithreading",
                    "Context Managers",
                    "Generators",
                ],
            },
        ),
    ),
)

# Add instruction phrases
config_builder.add_column(
    SamplerColumnConfig(
        name="instruction_phrase",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Write a function that",
                "Create a class that",
                "Implement a script",
                "Can you create a function",
                "Develop a module that",
            ],
        ),
    ),
)


## 🦜 Define LLM-Generated Columns

Now we'll set up the columns that will be generated by the LLMs, including the instruction and code implementation.


In [ ]:
# Generate instruction for the code
config_builder.add_column(
    LLMTextColumnConfig(
        name="instruction",
        model_alias=MODEL_ALIAS,
        system_prompt=(
            "You are an expert at generating clear and specific programming tasks."
        ),
        prompt=(
            "Generate an instruction to create Python code that solves a specific problem.\n"
            "Each instruction should begin with one of the following phrases: {{instruction_phrase}}.\n\n"
            "Important Guidelines:\n"
            "* Industry Relevance: Ensure the instruction pertains to the {{industry_sector}} sector and {{topic}} topic.\n"
            "* Code Complexity: Tailor the instruction to the {{code_complexity}} level. Utilize relevant {{code_concept}} where appropriate to match the complexity level.\n"
            "* Clarity and Specificity: Make the problem statement clear and unambiguous. Provide sufficient context to understand the requirements without being overly verbose.\n"
            "* Response Formatting: Do not include any markers such as ### Response ### in the instruction.\n"
        ),
    )
)

# Generate the Python code
config_builder.add_column(
    LLMCodeColumnConfig(
        name="code_implementation",
        model_alias=MODEL_ALIAS,
        code_lang=CodeLang.PYTHON,
        system_prompt=(
            "You are an expert Python programmer who writes clean, efficient, and well-documented code."
        ),
        prompt=(
            "Write Python code for the following instruction:\n"
            "Instruction: {{instruction}}\n\n"
            "Important Guidelines:\n"
            "* Code Quality: Your code should be clean, complete, self-contained, and accurate.\n"
            "* Code Validity: Please ensure that your Python code is executable and does not contain any errors.\n"
            "* Packages: Remember to import any necessary libraries, and to use all libraries you import.\n"
            "* Complexity & Concepts: The code should be written at a {{code_complexity}} level, making use of concepts such as {{code_concept}}.\n"
        ),
    )
)


## 🔍 Quality Assessment: LLM-as-a-Judge

When generating our synthetic dataset, we need to determine the quality of the generated data \
We use the LLM-as-a-Judge strategy to do this.

To do so, we need to define the rubric that the LLM should use to assess generation quality along with a prompt
that provides relavant instructions.


In [ ]:
TEXT_TO_PYTHON_JUDGE_TEMPLATE = """\
You are an expert in Python programming, with specialized knowledge in software engineering, data science, and algorithmic problem-solving. \
You think about potential flaws and errors in the code. You are a tough critic, but a fair one.

Take a deep breath and use the Python Code Quality Rubric below to score the **Generated Python Code** based on the INSTRUCTIONS.

#### INSTRUCTIONS
The Generated Python Code should be a valid response to the Natural Language Prompt below

Natural Language Prompt:
{{ instruction }}

Generated Python Code
{{ code_implementation }}
"""

python_scoring = [
    Score(
        name="Relevance",
        description="Adherence to INSTRUCTIONS and CONTEXT",
        options={
            "4": "Perfectly meets all specified requirements.",
            "3": "Meets most requirements with minor deviations.",
            "2": "Moderate deviation from the instructions.",
            "1": "Significant deviations from the instructions.",
            "0": "Does not adhere to the instructions.",
        },
    ),
    Score(
        name="Pythonic",
        description="Pythonic Code and Best Practices (Does the code follow Python conventions and best practices?)",
        options={
            "4": "The code exemplifies Pythonic principles, making excellent use of Python-specific constructs, standard library modules and programming idioms; follows all relevant PEPs.",
            "3": "The code closely follows Python conventions and adheres to many best practices; good use of Python-specific constructs, standard library modules and programming idioms.",
            "2": "The code generally follows Python conventions but has room for better alignment with Pythonic practices.",
            "1": "The code loosely follows Python conventions, with several deviations from best practices.",
            "0": "The code does not follow Python conventions or best practices, using non-Pythonic approaches.",
        },
    ),
    Score(
        name="Readability",
        description="Readability and Maintainability (Is the Python code easy to understand and maintain?)",
        options={
            "4": "The code is excellently formatted, follows PEP 8 guidelines, is elegantly concise and clear, uses meaningful variable names, ensuring high readability and ease of maintenance; organizes complex logic well. Docstrings are given in a Google Docstring format.",
            "3": "The code is well-formatted in the sense of code-as-documentation, making it relatively easy to understand and maintain; uses descriptive names and organizes logic clearly.",
            "2": "The code is somewhat readable with basic formatting and some comments, but improvements are needed; needs better use of descriptive names and organization.",
            "1": "The code has minimal formatting, making it hard to understand; lacks meaningful names and organization.",
            "0": "The code is unreadable, with no attempt at formatting or description.",
        },
    ),
    Score(
        name="Efficiency",
        description="Efficiency and Performance (Is the code optimized for performance?)",
        options={
            "4": "The solution is highly efficient, using appropriate data structures and algorithms; avoids unnecessary computations and optimizes for both time and space complexity.",
            "3": "The solution is efficient, with good use of Python's built-in functions and libraries; minor areas for optimization.",
            "2": "The solution is moderately efficient, but misses some opportunities for optimization; uses some inefficient patterns.",
            "1": "The solution shows poor efficiency, with notable performance issues; lacks effective optimization techniques.",
            "0": "The solution is highly inefficient; overlooks fundamental optimization practices, resulting in significant performance issues.",
        },
    ),
]

# Add an LLM judge to evaluate code quality
config_builder.add_column(
    LLMJudgeColumnConfig(
        name="code_judge_result",
        model_alias=MODEL_ALIAS,
        prompt=TEXT_TO_PYTHON_JUDGE_TEMPLATE,
        scores=python_scoring,
    )
)

## ⚡️ Quality Assessment: Code Validation

- NeMo Data Designer includes a built-in code validation feature that automatically checks the syntactic correctness and executable validity of \
  generated code snippets.

- This helps ensure that outputs from language models are not only syntactically correct, but also able to run successfully in the \
  intended programming language environment.

- Leveraging this validation step significantly increases dataset quality by promptly identifying invalid or non-functional code, \
  streamlining the process of generating reliable and production-ready data samples.

- NeMo Data Designer supports validation for these languages

  - Python (CodeLang.PYTHON)

  - SQL dialects:

    - ANSI SQL (CodeLang.SQL_ANSI)

    - MySQL (CodeLang.SQL_MYSQL)

    - PostgreSQL (CodeLang.SQL_POSTGRES)

    - SQLite (CodeLang.SQL_SQLITE)

    - T-SQL (CodeLang.SQL_TSQL)

    - BigQuery (CodeLang.SQL_BIGQUERY)


In [ ]:
config_builder.add_column(
    ValidationColumnConfig(
        name="code_validity_result",
        validator_type=ValidatorType.CODE,
        target_columns=["code_implementation"],  # Column containing the code
        validator_params=CodeValidatorParams(
            code_lang=CodeLang.PYTHON,
        ),
        batch_size=100,
    )
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [ ]:
# Preview a few records
preview = data_designer_client.preview(config_builder, num_records=2)

In [ ]:
# More previews
preview.display_sample_record()

### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [ ]:
# Print the analysis as a table.
preview.analysis.to_report()

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [ ]:
job_results = data_designer_client.create(config_builder, num_records=20)

# This will block until the job is complete.
job_results.wait_until_done()

In [ ]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

In [ ]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

In [ ]:
TUTORIAL_OUTPUT_PATH = "data-designer-tutorial-output"

# Download the job artifacts and save them to disk.
job_results.download_artifacts(
    output_path=TUTORIAL_OUTPUT_PATH,
    artifacts_folder_name="artifacts-community-contributions-text-to-code-text-to-python",
);